<a href="https://colab.research.google.com/github/Ossama-65/Gite/blob/main/Mode%CC%81lisation%2BCamemBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation des dépendances

Pour pouvoir exécuter ce Notebook, l'environnement de développement doit être bien configuré.

> Pour ce Notebook, on pourra utiliser <a href="https://web.eecs.umich.edu/~justincj/teaching/eecs442/WI2021/colab.html">Google Colab avec un GPU</a>.

In [ ]:
!pip install spacy transformers datasets sentencepiece seaborn pandas wordcloud tensorflow==2.11 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Ensuite, spaCy nous permettra de faire des opérations de traitement du texte adapté à la langue française.

In [ ]:
!python -m spacy download fr_core_news_sm

2023-04-22 14:58:37.655727: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 14:58:42.152766: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-22 14:58:42.153023: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-22 14:58:42.153055: W tensorfl

On termine par télécharger les données.

In [ ]:
!mkdir -p data/
!wget https://blent-keskia.s3.eu-west-3.amazonaws.com/poc/e-reputation/customer_service.csv -O data/customer_service.csv

--2023-04-22 14:59:05--  https://blent-keskia.s3.eu-west-3.amazonaws.com/poc/e-reputation/customer_service.csv
Resolving blent-keskia.s3.eu-west-3.amazonaws.com (blent-keskia.s3.eu-west-3.amazonaws.com)... 3.5.225.143
Connecting to blent-keskia.s3.eu-west-3.amazonaws.com (blent-keskia.s3.eu-west-3.amazonaws.com)|3.5.225.143|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9181328 (8.8M) [text/csv]
Saving to: ‘data/customer_service.csv’

data/customer_servi 100%[===================>]   8.76M  2.13MB/s    in 5.5s    

2023-04-22 14:59:11 (1.61 MB/s) - ‘data/customer_service.csv’ saved [9181328/9181328]



> Les cellules suivants reprennent les traitements effectués lors de la séance précédente.

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

data = pd.read_csv("data/customer_service.csv")
data

,Text,Class
0,C&#39;était génial.,1
1,J&#39;ai été client Comcast pendant plusieurs ...,0
2,C&#39;est la pire entreprise pour le service c...,0
3,"Comcast de Newtown, 139 N State Street, a augm...",0
4,Premier des mensonges mensonges mensonges. Qua...,0
...,...,...
7129,Des vents violents ont détruit ma ligne de câb...,1
7130,la nourriture n&#39;est pas savoureuse du tout...,0
7131,Wesley White a travaillé sans relâche et patie...,1
7132,"Ils ont déconnecté mon câble et Internet, car ...",0


In [ ]:
import regex as re

data_clean = data.copy()
# Résolution du problème de formatage du simple guillemet
data_clean["Text"] = data_clean["Text"].apply(lambda s: s.replace("&#39;", "'"))
# Tout en minuscule
data_clean["Text"] = data_clean["Text"].apply(lambda s: s.lower())
# Suppression des chiffres
data_clean["Text"] = data_clean["Text"].apply(lambda s: re.sub(r"\d*", "", s))
# Suppression des espaces en trop
data_clean["Text"] = data_clean["Text"].apply(lambda s: re.sub(r"\s+", " ", s))
# Suppression des ponctuations
data_clean["Text"] = data_clean["Text"].apply(lambda s: re.sub(r"[\.,\?\!]", "", s))
data_clean.head()

,Text,Class
0,c'était génial,1
1,j'ai été client comcast pendant plusieurs anné...,0
2,c'est la pire entreprise pour le service clien...,0
3,comcast de newtown n state street a augmenté m...,0
4,premier des mensonges mensonges mensonges quan...,0


In [ ]:
import spacy

nlp = spacy.load("fr_core_news_sm")

def stopwords(s):
    doc = nlp(s)
    filtered = ""
    for token in doc:
        if token.is_stop == False:
            filtered += token.text
            filtered += " "
    return filtered

data_clean["Text"] = data_clean["Text"].apply(lambda s: stopwords(s))

# Modélisation avec CamemBERT

<a href="https://camembert-model.fr/" target="_blank">CamemBERT</a> est un BERT (modèle de type Transformer) de HuggingFace entrainé avec des données en français pour classifier des phrases.

Avant d'entraîner le modèle, assurons-nous bien d'avoir un GPU à disposition avec la commande suivante.

In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


Si rien ne s'affiche, alors cela signifie que tu n'as pas de GPU à disposition, et que l'entraînement sera beaucoup trop long. 😢

Vérifions que le GPU est bien reconnu par TensorFlow.

In [ ]:
import tensorflow as tf

print("Nombre GPUs disponibles :", len(tf.config.list_physical_devices('GPU')))

Nombre GPUs disponibles : 0


### ➡️ À toi de jouer

En utilisant la fonction `train_test_split` de Scikit-Learn, séparer le jeu de données en deux sous-ensembles d'entraînement et de test : `X_train`, `X_test`, `y_train` et `y_test`.

In [ ]:
from sklearn.model_selection import train_test_split

# On sépare le jeu de données en sous-ensembles d'entraînement et de test
# TODO
X_train, X_test, Y_train, Y_test = train_test_split( list(data_clean["Text"]), list(data_clean["Class"]), test_size = 0.15, random_state = 42 )
#X_train, X_test, Y_train, Y_test = train_test_split(data, data, test_size=0.2, random_state=42)


Pour entraîner notre modèle, nous allons faire du **transfer learning** : nous allons nous baser sur un modèle déjà entraîné pour un usage général, afin de le spécifier pour notre case d'usage.

In [ ]:
import transformers

from transformers import AutoTokenizer, AutoModelForMaskedLM

# Import du tokenizer et le modèle avec le paramètre num_labels=2 pour modifier la dernière couche dense de sortie
tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = transformers.TFCamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=2)

All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

Some layers of TFCamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Par défaut, BERT nécessite un format de données un peu particulier, que l'on utilise avec la fonction suivante.

In [ ]:
# Tokeniser et transformer les données en "dataset" convenable au BERT (avec la bibliothèque Dataset de HuggingFace), soit un dict de tensors
def get_train_ds(X_train, Y_train, batch_size=8):
    train_encodings = tokenizer(X_train, truncation=True, padding=True)
    train_ds = tf.data.Dataset.from_tensor_slices((dict(train_encodings), Y_train))
    train_ds = train_ds.batch(batch_size)
    return train_ds

train_ds = get_train_ds(X_train, Y_train)

Nous définissons ensuite les hyper-paramètres pour contrôler en amont l'apprentissage du modèle.

### ➡️ À toi de jouer

Compiler le modèle TensorFlow en choisissant une fonction de perte `SparseCategoricalCrossentropy`, l'optimiseur définie dans la cellule de code et une métrique `SparseCategoricalAccuracy`.

In [ ]:
# Définition des hyperparamètres et compilation le modèle
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=5e-5, decay_steps=10000, decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=optimizer,
  metrics=tf.metrics.SparseCategoricalAccuracy()
)


Maintenant, il ne reste plus qu'à entraîner le modèle.

> Avec 3 itérations, il faut s'attendre à environ 1 heure d'apprentissage avec un GPU.

In [ ]:
history = model.fit(train_ds, epochs=3)

Epoch 1/3
109/758 [===>..........................] - ETA: 11:44:00 - loss: 0.3750 - sparse_categorical_accuracy: 0.8635

Comme nous pouvons l'observer, l'accuracy est plutôt satisfaisante. On enregistre ensuite les fichiers de poids du tokenizer et du modèle BERT.

In [ ]:
tokenizer.save_pretrained("bert_tokenizer")
model.save_pretrained("bert_model")

# Inférence

Maintenant, essayons de charger notre modèle.

In [ ]:
from transformers import TextClassificationPipeline

model_path = "bert_model"
tokenizer = AutoTokenizer.from_pretrained("bert_tokenizer")
model = transformers.TFCamembertForSequenceClassification.from_pretrained(
    model_path,
    id2label={0: "La personne est mécontente", 1: "La personne a donné un avis neutre ou positif"}
)

### ➡️ À toi de jouer

En utilisant l'objet `TextClassificationPipeline`, prédit le label et le score associé à chaque phrase.

In [ ]:
text = ["Au bout d'un moment il faudrait commencer à régler ce problème de chauffage à l'école, les enfants ont trop froid !"]
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)
pipe(text)

In [ ]:
text = ["C'est une bonne idée ce panier anti-inflation, mais il faudrait peut être commencer à regarder du côté des taxes de la ville"]
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)
pipe(text)

# Analyse lexicale

### ➡️ À toi de jouer

À l'aide de `displacy`, afficher les relations lexicales entre les mots.

In [ ]:
from spacy import displacy

doc = nlp("Au bout d'un moment il faudrait commencer à régler ce problème de chauffage à l'école, les enfants ont trop froid !")
displacy.render(doc, style="dep", jupyter=True)

In [ ]:
doc = nlp("Quand vous allez réparer le feu qui marche pas rue de versailles ? ça fait presque 6 mois que c'est comme ça !")
displacy.render(doc, style="dep", jupyter=True)

## Extraction des topics

Dans cette partie, nous allons extraire les mots-clés de chaque topic.

In [ ]:
import string

from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF 

print("Extracting the tf-idf features form NMF")
tfidf_vectorizer = TfidfVectorizer(max_df = 0.5, min_df = 5, max_features = 50, ngram_range = (1,4))
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_clean['Text'])
print("done in %0.3fs." % (time() - t0))

In [ ]:
# nombre de clusters/topics
n_comp = [1,3,5,7,9]
# perte/erreur('Reconstruction Error - Frobenius Norm')
loss = []

for comps in n_comp:
  t0 = time()
  nmf = NMF(n_components = comps , random_state = 1, beta_loss = 'kullback-leibler',solver = 'mu',max_iter = 200,alpha_W = 0.1, l1_ratio = 0.5).fit(tfidf)
  loss.append(nmf.reconstruction_err_)
  print ("done in %0.3f " % (time() -t0))

In [ ]:
plt.plot(loss)
plt.xlabel('Number of Topics')
plt.ylabel('Reconstruction Error - Frobenius Norm')

In [ ]:
def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
    message = "Topic #%d: " % topic_idx
    message += " ".join([feature_names[i]
                         for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)
    print()

In [ ]:
print_top_words(nmf, tfidf_vectorizer.get_feature_names(), 10)